In [ ]:
from pathlib import Path
import sys
from dotenv import load_dotenv

from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatPerplexity
from langchain_mistralai import ChatMistralAI
from langchain_cohere import ChatCohere


from langchain.agents import AgentExecutor
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)

from langchain_core.runnables import ConfigurableField
from langchain_core.runnables import Runnable


load_dotenv(".env")

In [ ]:
llm = ChatAnthropic(
    model="claude-3-opus-20240229",
    max_tokens=2000,
    temperature=0.9,
    # anthropic_api_key=os.environ.get("ANTHROPIC_API_KEY", "not_provided"),
    streaming=True,
    verbose=True,
).configurable_alternatives(  # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="llm"),
    # default_key="openai_gpt_4_turbo_preview",
    default_key="anthropic_claude_3_opus",
    anthropic_claude_3_5_sonnet=ChatAnthropic(
        model="claude-3-5-sonnet-20240620",
        max_tokens=2000,
        temperature=0.9,
        # anthropic_api_key=os.environ.get("ANTHROPIC_API_KEY", "not_provided"),
        streaming=True,
        stream_usage=True,
        verbose=True,
    ),
    openai_gpt_3_5_turbo_1106=ChatOpenAI(
        model="gpt-3.5-turbo-1106",
        verbose=True,
        streaming=True,
        temperature=0.9,
    ),
    openai_gpt_4_turbo_preview=ChatOpenAI(
        temperature=0.9,
        model="gpt-4-turbo-preview",
        verbose=True,
        streaming=True,
    ),
    openai_gpt_4o=ChatOpenAI(
        temperature=0.9,
        model="gpt-4o",
        verbose=True,
        streaming=True,
    ),
    openai_gpt_4o_mini=ChatOpenAI(
        temperature=0.9,
        model="gpt-4o-mini",
        verbose=True,
        streaming=True,
    ),
    pplx_sonar_medium_chat=ChatPerplexity(
        model="sonar-medium-chat", temperature=0.9, verbose=True, streaming=True
    ),
    mistral_large=ChatMistralAI(
        model="mistral-large-latest", temperature=0.1, verbose=True, streaming=True
    ),
    command_r_plus=ChatCohere(
        model="command-r-plus", temperature=0.9, verbose=True, streaming=True
    ),
)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import tool


@tool
def news_extractor(title: str, content: str, src_url: str) -> str:
    """
    Extract and analyze the content of a news article.

    This function uses a predefined prompt template and language model to analyze
    a news article, providing key information extraction, sentiment analysis,
    topic classification, relevance analysis, and impact assessment.

    Parameters:
    title (str): The title of the news article
    content (str): The main body content of the news article
    src_url (str): The source URL of the news article

    Returns:
    str: A string containing the analysis results, including key information,
         sentiment analysis, topic classification, relevance, and impact assessment
    """
    with open("news_analysis_test_prompt.txt", "r", encoding="utf-8") as file:
        prompt_template = file.read()
    chain = ChatPromptTemplate.from_template(prompt_template) | llm | StrOutputParser()
    result = chain.invoke(
        {
            "title": title,
            "content": content,
            "source_url": src_url,
        },
        config={"configurable": {"model": "anthropic_claude_3_5_sonnet"}},
    )
    return result

In [ ]:
def create_agent_executor(llm_agent: Runnable) -> AgentExecutor:
    # 从文件加载系统提示
    with open("system_news_analysis_test_prompt.txt", "r", encoding="utf-8") as file:
        system_message = file.read()

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_message),
            MessagesPlaceholder(variable_name="chat_history"),
            # SystemMessagePromptTemplate.from_template(
            #     "If using the search tool, prefix the string parameter with [S]."
            # ),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )

    from custom_agent_excutor import CustomToolCallingAgentExecutor
    from tools_basic import tools

    tools += [news_extractor]
    executor = CustomToolCallingAgentExecutor(
        llm=llm_agent, prompts=prompt, tools=tools
    )
    return executor

In [ ]:
executor = create_agent_executor(llm_agent=llm)

In [ ]:
news = """The news as follow:
Title: BitGo计划2025年1月推出创新型稳定币USDS
Content: 加密货币托管公司BitGo宣布计划于2025年1月推出一种名为USDS的美元支持稳定币。该稳定币的独特之处在于将为提供流动性的机构提供奖励。

BitGo CEO Mike Belshe在Token2049大会前接受CoinDesk采访时表示，USDS将由短期国债、隔夜回购协议和现金支持，类似于市场上的其他稳定币。但与众不同的是，USDS被称为首个"开放参与"的稳定币。

USDS将通过将储备金产生的部分回报分配给提供流动性的机构来激励它们。BitGo计划在所有主要交易所上市USDS，并目标在明年此时达到100亿美元的资产规模。

Source URL:https://x.com/LayerZero_Core/status/1836037465884577894

请分析一下这则新闻快讯。
"""

streams = executor.astream_events(
    {"input": news, "chat_history": []},
    config={
        "configurable": {"llm": "anthropic_claude_3_5_sonnet"},
    },
    version="v1",
)
buff = ""
async for chunk in streams:
    if chunk["event"] == "on_chat_model_stream":
        if isinstance(chunk["data"]["chunk"].content, str):
            buff += chunk["data"]["chunk"].content
        elif (
            isinstance(chunk["data"]["chunk"].content, list)
            and len(chunk["data"]["chunk"].content) > 0
            and "text" in chunk["data"]["chunk"].content[0]
        ):
            buff += chunk["data"]["chunk"].content[0]["text"]
    print(chunk)
print(buff)